# MACHINE LEARNING I - PCA

## 1. Librerias y Datos

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling
import io
import warnings
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('data.csv')
data.head()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,...,meal_FB,meal_HB,meal_SC,meal_Undefined,deposit_type_Non Refund,deposit_type_Refundable,reserverd/assigned,customer_type_Group,customer_type_Transient-Party,stays_nights
0,0,3.018292,17.0,20,2,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,4
1,0,6.724848,17.0,17,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,7.729769,8.0,16,2,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,7
3,1,7.779122,30.0,23,2,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,4
4,0,5.366444,33.0,18,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,3


Separamos la variable objetivo (is_canceled) de las demás.

In [14]:
x = data.iloc[:,1:26].values
y = data.iloc[:,0].values

## NORMALIZAMOS

In [16]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(x)